In [1]:
import pandas as pd
import pymysql

###### 1. 환전신청 테이블 가져오기
'''
아테나 포인트 차감 로그 조회 쿼리
select memberid, type, description, point, year, month, day
from "partners_point_log"  
WHERE year = '2022'
AND   concat(month, day) >='0626'
AND   concat(month, day) <='0723'
AND   type = 'EXCHANGE'
'''

In [2]:
ex = pd.read_csv("exchange_220902.csv")

In [3]:
ex.head()

,memberid,type,description,point,year,month,day
0,27406,EXCHANGE,출금 신청,45000.0,2022,8,31
1,1353,EXCHANGE,출금 신청,200000.0,2022,8,31
2,53758,EXCHANGE,출금 신청,65000.0,2022,8,31
3,26228,EXCHANGE,출금 신청,45000.0,2022,8,31
4,41270,EXCHANGE,출금 신청,45000.0,2022,8,31


In [4]:
#복제디비 가져오기
connect = pymysql.connect(host='newspic-replica.c8oi6pnysk97.ap-northeast-2.rds.amazonaws.com'
                        , port = 3306
                        , user='newspic'
                        , password ='pic0901news'
                        , db = 'newspic'
                        , charset='utf8')
# connection으로부터 cursor 생성
cur = connect.cursor()

In [5]:
query = '''
      select m.no, m.uname, m.requestKey
      from   member m
	  where  m.level in (60,61,65)
            '''
member = pd.read_sql(query, connect)

In [6]:
ex2 = pd.merge(member, ex,  left_on='no', right_on='memberid')

In [7]:
ex2.drop(['no'], axis=1, inplace=True)

In [8]:
ex2.head()

,uname,requestKey,memberid,type,description,point,year,month,day
0,장지환,d6nqd43U,381,EXCHANGE,출금 신청,7820000.0,2022,8,31
1,장지환,d6nqd43U,381,EXCHANGE,출금 신청,1330000.0,2022,7,13
2,장지환,d6nqd43U,381,EXCHANGE,출금 신청,320000.0,2022,8,3
3,장지환,d6nqd43U,381,EXCHANGE,출금 신청,2370000.0,2022,8,3
4,장지환,d6nqd43U,381,EXCHANGE,출금 신청,130000.0,2022,7,27


In [9]:
# member 테이블 조회
member.head()

,no,uname,requestKey
0,113,오상헌,f2Pfa95D
1,377,del377z3Upe06c,z3Upe06c
2,378,김채린,d5Ddr72j
3,380,김수민,W6fsy57f
4,381,장지환,d6nqd43U


In [10]:
# 가입포인트 테이블 가져오기
query = '''
SELECT m.uname, m.requestKey, A.*
FROM (select m.no, m.uname, m.requestKey
      from   member m
	 where  m.level in (60,61,65)) m inner join tbPartnersUserSubPointLog A on m.no= A.memberId
     WHERE A.updateDate BETWEEN '2022-01-01' AND '2022-07-23'
            '''
point = pd.read_sql(query, connect)

In [11]:
# 포인트 테이블 조회
point.head()

,uname,requestKey,no,memberId,type,description,point,usablePoint,createdDate,updateDate,expireDate,expireYn
0,박성근,A6mit98P,1,2070,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,0.0,2021-09-01 11:43:36,2022-07-01 00:30:20,2022-07-01,Y
1,del2071A0CoP76o,A0CoP76o,2,2071,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,0.0,2021-09-01 13:18:34,2022-07-01 00:30:20,2022-07-01,Y
2,del2072p0nox59F,p0nox59F,3,2072,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,0.0,2021-09-01 13:25:08,2022-07-01 00:30:20,2022-07-01,Y
3,송유경,v9zvs18t,5,2074,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,0.0,2021-09-01 13:28:19,2022-07-01 00:30:20,2022-07-01,Y
4,del2075P7GLZ92q,P7GLZ92q,6,2075,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,0.0,2021-09-01 13:31:48,2022-07-01 00:30:20,2022-07-01,Y


In [12]:
# 열 no컬럼 삭제 
point.drop(['no'],axis=1, inplace=True)

In [13]:
# 포인트 테이블 정보 조회
point.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16038 entries, 0 to 16037
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uname        16038 non-null  object        
 1   requestKey   16038 non-null  object        
 2   memberId     16038 non-null  int64         
 3   type         16038 non-null  object        
 4   description  16038 non-null  object        
 5   point        16038 non-null  float64       
 6   usablePoint  16038 non-null  float64       
 7   createdDate  16038 non-null  datetime64[ns]
 8   updateDate   16038 non-null  datetime64[ns]
 9   expireDate   16038 non-null  object        
 10  expireYn     16038 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 1.3+ MB


In [14]:
# 포인트 사용월 컬럼 생성
point['month'] = point['updateDate'].dt.month 

In [15]:
#point[point['expireYn']=='P']

In [16]:
point = point[['uname','requestKey','memberId','type','description','point','month']]
point

,uname,requestKey,memberId,type,description,point,month
0,박성근,A6mit98P,2070,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7
1,del2071A0CoP76o,A0CoP76o,2071,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7
2,del2072p0nox59F,p0nox59F,2072,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7
3,송유경,v9zvs18t,2074,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7
4,del2075P7GLZ92q,P7GLZ92q,2075,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7
...,...,...,...,...,...,...,...
16033,한채옥,D1tIc88a,54021,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7
16034,김승기,k0sig42A,54369,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7
16035,성효영,y2CGn92O,54391,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7
16036,유상민,d5asK60G,54848,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7


In [17]:
#point[point['requestKey']=='a5EJc15N']

In [18]:
# 열 지정하여 이름변경
point.rename(columns={'memberId':'memberid'},inplace=True)

In [19]:
#위아래로 합치기 행 기준
exchange = pd.concat([point, ex2], axis=0)

In [20]:
exchange

,uname,requestKey,memberid,type,description,point,month,year,day
0,박성근,A6mit98P,2070,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7,NaN,NaN
1,del2071A0CoP76o,A0CoP76o,2071,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7,NaN,NaN
2,del2072p0nox59F,p0nox59F,2072,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7,NaN,NaN
3,송유경,v9zvs18t,2074,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7,NaN,NaN
4,del2075P7GLZ92q,P7GLZ92q,2075,JOIN_EVENT,신규 가입 이벤트 포인트,5000.0,7,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2339,김환진,Z7Fxp27l,59482,EXCHANGE,출금 신청,45000.0,8,2022.0,31.0
2340,정민우,t3DGv35Y,59534,EXCHANGE,출금 신청,195000.0,8,2022.0,31.0
2341,정민우,t3DGv35Y,59534,EXCHANGE,출금 신청,200000.0,8,2022.0,31.0
2342,최초혜,O5wFl27x,60301,EXCHANGE,출금 신청,45000.0,8,2022.0,31.0


In [22]:
exchange.to_csv("환전신청포인트_220902.csv",encoding='utf-8-sig')